<a href="https://colab.research.google.com/github/TeresaGracia/AprendizajeReforzado/blob/main/TSP_20220206.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
pip install or_gym

In [45]:
#Implementación Reinforce
import numpy as np
import torch
from matplotlib import pyplot as plt
import random
import or_gym

In [46]:
#Creación red neuronal
env=or_gym.make('TSP-v1')
env.reset()
obs_size = env.observation_space.shape[0] 
n_actions = env.action_space.n  
HIDDEN_SIZE = 256

model = torch.nn.Sequential(
             torch.nn.Linear(obs_size, HIDDEN_SIZE),
             torch.nn.ReLU(),
             torch.nn.Linear(HIDDEN_SIZE, n_actions),
             torch.nn.Softmax(dim=0)
     )


In [47]:
#Ascenso del gradiente
LEARNING_RATE = 0.0018
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [48]:
#Bucle de entrenamiento
HORIZON = 5000
MAX_TRAJECTORIES =2500
GAMMA = 1

score = [] 
for trajectory in range(MAX_TRAJECTORIES):
    current_state = env.reset()
    done = False
    transitions = [] 
    reward_transitions = [] 
    reward_batch=[] 

    for t in range(HORIZON):
        actions_prob = model(torch.from_numpy(current_state).float())
        p=actions_prob.data.numpy()
        longitud=len(p)
        a=[]
        for j in range(int(env.N)):#Para obtener la dimensión del espacio acción
          a.append(j)        
        action = np.random.choice(a,p=actions_prob.data.numpy()) 
        contador=0
        while current_state[action+1]==1:
          action=np.random.choice(a,p=actions_prob.data.numpy()) 
          if contador==50:
            break
          else:
            contador=contador+1        
        previous_state = current_state
        current_state, reward, done, info = env.step(action) 
        if reward!=-100 and reward!=1000: #Para que la recompensa sea mayor a menor distancia
          reward=1-reward
        transitions.append((previous_state, action ,t+1)) 
        reward_transitions.append((reward)) 
        if done: 
            break
    score.append(len(transitions))
    batch_Gvals =[]
    power=0
    new_Gval=0
    for recompensa in reward_transitions:
      new_Gval=new_Gval+((GAMMA**power)*recompensa)
      power+=1
    batch_Gvals.append(new_Gval)
    expected_returns_batch=torch.FloatTensor(batch_Gvals)  
    expected_returns_batch /= expected_returns_batch.max()

    state_batch = torch.Tensor([s for (s,a,r) in transitions]) 
    action_batch = torch.Tensor([a for (s,a,r) in transitions]) 

    predicted_batch = model(state_batch) 
    prob_batch = predicted_batch.gather(dim=1,index=action_batch.long().view(-1,1)).squeeze() #Creo que esto es lo que está mal



    
    loss = - torch.sum(torch.log(prob_batch) * expected_returns_batch) 
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if trajectory % 100 == 0 and trajectory>0:
        print('Trajectory {}\tAverage Score: {:.2f}'.format(trajectory, np.mean(score[-50:-1])))


Trajectory 100	Average Score: 45.98
Trajectory 200	Average Score: 47.37
Trajectory 300	Average Score: 47.47
Trajectory 400	Average Score: 48.16
Trajectory 500	Average Score: 47.78
Trajectory 600	Average Score: 48.65
Trajectory 700	Average Score: 48.96
Trajectory 800	Average Score: 48.94
Trajectory 900	Average Score: 48.94
Trajectory 1000	Average Score: 48.84
Trajectory 1100	Average Score: 48.69
Trajectory 1200	Average Score: 48.45
Trajectory 1300	Average Score: 49.00
Trajectory 1400	Average Score: 49.00
Trajectory 1500	Average Score: 48.96
Trajectory 1600	Average Score: 49.00
Trajectory 1700	Average Score: 49.00
Trajectory 1800	Average Score: 49.00
Trajectory 1900	Average Score: 49.00
Trajectory 2000	Average Score: 49.00
Trajectory 2100	Average Score: 49.00
Trajectory 2200	Average Score: 49.00
Trajectory 2300	Average Score: 49.00
Trajectory 2400	Average Score: 49.00


In [49]:
#Aplicación de la red neuronal para resolver el entorno

def watch_agent():
  env = or_gym.make('TSP-v1')
  state = env.reset()
  rewards = []
  win=0
  lost=0
  total=0
  max_win=0
  max_lost=-100
  for i in range(50000):
    rewards=[]
    state = env.reset()
    for t in range(2000):
      pred = model(torch.from_numpy(state).float())
      action = np.random.choice(a, p=pred.data.numpy())
      state, reward, done, _ = env.step(action)
      if reward!= -100 and reward!=1000:
        reward=1-reward
      rewards.append(reward)
      if done:
          #print(state)
          #print("Reward:", sum([r for r in rewards]))
          total_reward=sum([r for r in rewards])
          if total_reward>0:
            win=win+1
            total=total+1
            if total_reward>max_win:
              max_win=total_reward
          else:
            lost=lost+1
            total=total+1
            if total_reward>max_lost:
              max_lost=total_reward
          break 
  env.close()
  print("Completados",win)
  print("Perdidos",lost)
  print("Total", total)
  print("Porcentaje completados",100*win/total)
  print("Máxima recompensa en los completados",max_win)
  print("Máxima recompensa en los perdidos",max_lost)
watch_agent()

Completados 41876
Perdidos 8124
Total 50000
Porcentaje completados 83.752
Máxima recompensa en los completados 29.788704032718424
Máxima recompensa en los perdidos -71.95193091601143
